In [1]:
import numpy as np
import cv2

In [2]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [3]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
    
	vs, us = np.where(seg != 0)
	zs = dpt[vs, us]
	rhs = np.concatenate((us.reshape(-1,1), vs.reshape(-1,1), np.ones((us.shape[0],1))), axis=1, dtype=np.float32) * zs.reshape(-1,1)

	pc = np.dot(rhs, np.linalg.inv(K.T))

	return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [4]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
one_way_CD = 0
distance_matrix = np.expand_dims(partial_pc_sampled,0) - np.expand_dims(full_pc_sampled,1)
distance_matrix = np.linalg.norm(distance_matrix, axis=-1)
one_way_CD = np.mean(np.min(distance_matrix, axis=0))

print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', np.array([one_way_CD]))

one way chamfer distance:  0.009987344849640437
